In [ ]:
# pip install --upgrade    langchain langchain-huggingface sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

df = pd.read_csv("dataset/Gita_questions.csv", usecols=["chapter", "verse","translation"])
df.head()

,chapter,verse,translation
0,1,1,"Dhritarashtra said, ""What did my people and th..."
1,1,2,Sanjaya said: Having seen the army of the Pand...
2,1,3,"Behold, O Teacher! This mighty army of the son..."
3,1,4,"Here are heroes, mighty archers, equal in batt..."
4,1,5,"Dhrishtaketu, Chekitana, the valiant king of K..."


In [3]:
# Build one sentence per row (include speaker if you want)
df["content"] = (
    "Chapter " + df["chapter"].astype(int).astype(str)
    + " Verse " + df["verse"].astype(int).astype(str)
    + " — " + df["translation"].astype(str).str.strip()
)

In [4]:
df.head()

,chapter,verse,translation,content
0,1,1,"Dhritarashtra said, ""What did my people and th...","Chapter 1 Verse 1 — Dhritarashtra said, ""What ..."
1,1,2,Sanjaya said: Having seen the army of the Pand...,Chapter 1 Verse 2 — Sanjaya said: Having seen ...
2,1,3,"Behold, O Teacher! This mighty army of the son...","Chapter 1 Verse 3 — Behold, O Teacher! This mi..."
3,1,4,"Here are heroes, mighty archers, equal in batt...","Chapter 1 Verse 4 — Here are heroes, mighty ar..."
4,1,5,"Dhrishtaketu, Chekitana, the valiant king of K...","Chapter 1 Verse 5 — Dhrishtaketu, Chekitana, t..."


In [5]:

loader = DataFrameLoader(
    df,
    page_content_column="content",
)

# documents = loader.load()
# documents[0]
# all_doc = []
# for doc in loader.lazy_load():
#     all_doc.append(doc.page_content)
    # print(doc.metadata)
docs = list(loader.lazy_load())


In [6]:
# docs = list(loader.lazy_load())  # each is a Document with page_content + metadata

group_size = 6  # 5–8
grouped_docs = []
for i in range(0, len(docs), group_size):
    group = docs[i:i + group_size]
    content = "\n\n".join(d.page_content for d in group)
    # Keep useful metadata; here we preserve the chapter/verse lists
    grouped_docs.append(
        Document(
            page_content=content,
            metadata={
                "row_start": i,
                "row_end": i + len(group) - 1,
                "chapters": [int(d.metadata.get("chapter")) for d in group if d.metadata.get("chapter") is not None],
                "verses": [int(d.metadata.get("verse")) for d in group if d.metadata.get("verse") is not None],
            },
        )
    )

# Optional: further split overly long chunks by characters
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, separators=["\n\n", "\n", " "]
)
final_docs = splitter.split_documents(grouped_docs)
final_docs

[Document(metadata={'row_start': 0, 'row_end': 5, 'chapters': [1, 1, 1, 1, 1, 1], 'verses': [1, 2, 3, 4, 5, 6]}, page_content='Chapter 1 Verse 1 — Dhritarashtra said, "What did my people and the sons of Pandu do when they had assembled together, eager for battle, on the holy plain of Kurukshetra, O Sanjaya?"\n\nChapter 1 Verse 2 — Sanjaya said: Having seen the army of the Pandavas drawn up in battle array, King Duryodhana approached his teacher, Drona, and spoke these words.\n\nChapter 1 Verse 3 — Behold, O Teacher! This mighty army of the sons of Pandu, arrayed by the son of Drupada, thy wise disciple.\n\nChapter 1 Verse 4 — Here are heroes, mighty archers, equal in battle to Bhima and Arjuna, Yuyudhana (Satyaki), Virata, and Drupada—all mighty warriors.\n\nChapter 1 Verse 5 — Dhrishtaketu, Chekitana, the valiant king of Kasi, Purujit, Kuntibhoja, and Saibya—the best of men.\n\nChapter 1 Verse 6 — The strong Yudhamanyu and the brave Uttamaujas, the son of Subhadra (Abhimanyu, the son 

In [9]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [14]:

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [d.page_content for d in final_docs]  # final_docs: List[Document]
vectors = embedding_model.embed_documents(texts)
len(vectors), len(vectors[0])

(117, 384)

In [23]:
# Pip install necessary package
%pip install -qU  neo4j
%pip install -qU  langchain-openai langchain-neo4j
%pip install -qU  tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires kubernetes>=28.1.0, which is not installed.
chromadb 0.5.23 requires opentelemetry-api>=1.2.0, which is not installed.
chromadb 0.5.23 requires opentelemetry-exporter-otlp-proto-grpc>=1.2.0, which is not installed.
chromadb 0.5.23 requires opentelemetry-instrumentation-fastapi>=0.41b0, which is not installed.
chromadb 0.5.23 requires opentelemetry-sdk>=1.2.0, which is not installed.
chromadb 0.5.23 requires pypika>=0.48.9, which is not installed.
embedchain 0.1.126 requires beautifulsoup4<5.0.0,>=4.12.2, which is not installed.
embedchain 0.1.126 requires google-cloud-aiplatform<2.0.0,>=1.26.1, which is not installed.
embedchain 0.1.126 requires gptcache<0.2.0,>=0.1.43, which is not installed.
embedchain 0.1.126 requires langchain-cohere<0.4.0,>=0.3.0, which is not installed.
camelot-py 1.

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.126 requires beautifulsoup4<5.0.0,>=4.12.2, which is not installed.
embedchain 0.1.126 requires google-cloud-aiplatform<2.0.0,>=1.26.1, which is not installed.
embedchain 0.1.126 requires gptcache<0.2.0,>=0.1.43, which is not installed.
embedchain 0.1.126 requires langchain-cohere<0.4.0,>=0.3.0, which is not installed.
embedchain 0.1.126 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.33 which is incompatible.
embedchain 0.1.126 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.28 which is incompatible.
embedchain 0.1.126 requires pypdf<6.0.0,>=5.0.0, but you have pypdf 6.0.0 which is incompatible.
embedchain 0.1.126 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.11.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[

In [ ]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jVector

load_dotenv()  # expects NEO4J_URL, NEO4J_USERNAME, NEO4J_PASSWORD in .env

INDEX_NAME = "gita_chunks_idx"     # change if you switch embedding models (dims differ)
NODE_LABEL = "Chunk"
TEXT_PROP  = "text"
EMB_PROP   = "embedding"

# final_docs: List[Document] you created after grouping/splitting rows
vectorstore = Neo4jVector.from_documents(
    documents=final_docs,
    embedding=embedding_model,
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME") or "neo4j",
    password=os.getenv("NEO4J_PASSWORD"),
    index_name=INDEX_NAME,
    node_label=NODE_LABEL,
    text_node_property=TEXT_PROP,
    embedding_node_property=EMB_PROP,
)
print("Ingested", len(final_docs), "documents into Neo4j vector store:", INDEX_NAME)

Ingested 117 documents into Neo4j vector store: gita_chunks_idx


In [25]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

query = "Why did Arjuna ask Krishna to place the chariot between the armies?"
results = retriever.get_relevant_documents(query)

for i, d in enumerate(results, start=1):
    print(f"\nTop {i}:\n{d.page_content[:400]}...")
    print("Metadata:", d.metadata)

C:\Users\parth\AppData\Local\Temp\ipykernel_4328\3884556703.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



Top 1:
Chapter 1 Verse 19 — The tumultuous sound rent the hearts of Dhritarashtra's party, reverberating through both heaven and earth.

Chapter 1 Verse 20 — Then, seeing the people of Dhritarashtra's party standing arrayed and the discharge of weapons about to begin, Arjuna, the son of Pandu whose ensign was a monkey, took up his bow and said the following to Krishna, O Lord of the Earth.

Chapter 1 Ver...
Metadata: {'row_end': 23, 'verses': [19, 20, 21, 22, 23, 24], 'row_start': 18, 'chapters': [1, 1, 1, 1, 1, 1]}

Top 2:
Chapter 1 Verse 25 — In front of Bhishma and Drona, and all the rulers of the earth, he said: "O Arjuna, son of Pritha, behold these Kurus gathered together."

Chapter 1 Verse 26 — Then, Arjuna (son of Pritha) saw there (in the armies) stationed fathers, grandfathers, teachers, maternal uncles, brothers, sons, grandsons, and friends.

Chapter 1 Verse 27 — He saw fathers-in-law and friends in both ...
Metadata: {'row_end': 29, 'verses': [25, 26, 27, 28, 29, 30], 'ro

In [ ]:
# pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# If needed:
# %pip install -U langchain-huggingface sentence-transformers langchain-neo4j python-dotenv

import os
import re
from dotenv import load_dotenv

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_neo4j import Neo4jVector

load_dotenv()  # expects NEO4J_URL, NEO4J_USERNAME, NEO4J_PASSWORD, groq_api_key

# Embeddings (same model used to ingest vectors)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# LLM (Groq per your README)
llm = ChatGroq(model="gemma2-9b-it", temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"))

INDEX_NAME = "gita_chunks_idx"  # must match what you used when ingesting
NODE_LABEL = "Chunk"
TEXT_PROP  = "text"
EMB_PROP   = "embedding"

# Reconnect to existing Neo4j vector index
vectorstore = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME") or "neo4j",
    password=os.getenv("NEO4J_PASSWORD"),
    index_name=INDEX_NAME,
    node_label=NODE_LABEL,
    text_node_property=TEXT_PROP,
    embedding_node_property=EMB_PROP,
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 8})

In [18]:
from typing import List
from langchain_core.documents import Document

def extract_refs(docs: List[Document]) -> List[str]:
    """
    Extract unique chapter:verse references from Document metadata or content.
    Prioritizes metadata (chapter/verse or chapters/verses), falls back to regex.
    """
    refs = []

    for d in docs:
        # Prefer list-form metadata created during grouping
        ch_list = d.metadata.get("chapters")
        vs_list = d.metadata.get("verses")
        if isinstance(ch_list, list) and isinstance(vs_list, list) and len(ch_list) == len(vs_list) and len(ch_list) > 0:
            for ch, vs in zip(ch_list, vs_list):
                try:
                    refs.append(f"Chapter {int(ch)} Verse {int(vs)}")
                except Exception:
                    pass
            continue

        # Fallback: single chapter/verse in metadata
        ch = d.metadata.get("chapter")
        vs = d.metadata.get("verse")
        if ch is not None and vs is not None:
            try:
                refs.append(f"Chapter {int(ch)} Verse {int(vs)}")
                continue
            except Exception:
                pass

        # Fallback: parse from the content line "Chapter X Verse Y — ..."
        m = re.search(r"Chapter\s+(\d+)\s+Verse\s+(\d+)", d.page_content)
        if m:
            refs.append(f"Chapter {int(m.group(1))} Verse {int(m.group(2))}")

    # Deduplicate preserving order
    seen = set()
    unique_refs = []
    for r in refs:
        if r not in seen:
            seen.add(r)
            unique_refs.append(r)
    return unique_refs

def format_context(docs: List[Document], max_chars: int = 6000) -> str:
    """
    Join doc page_contents into a single context string, capped in length to keep prompt manageable.
    """
    buf = []
    total = 0
    for d in docs:
        chunk = d.page_content.strip()
        if not chunk:
            continue
        if total + len(chunk) > max_chars:
            break
        buf.append(chunk)
        total += len(chunk)
    return "\n\n".join(buf)

In [21]:
import re
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.documents import Document

# Reuse your existing `retriever`, `llm`, and helpers `format_context(docs)` and `extract_refs(docs)`

parser = JsonOutputParser()

# We explicitly instruct the model to ONLY produce the JSON field "text".
# We will inject the authoritative references ourselves from retrieved docs.
format_instructions = """Return ONLY a JSON object with this exact shape:
{
  "text": "<concise answer in 3–5 sentences>"
}
Do not include any other keys, no markdown, and no trailing text outside JSON.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Use the provided context from the Bhagavad Gita to answer "
     "the user's question accurately and concisely in 3–5 sentences. "
     "Do not include verse text in your answer. If the context is insufficient, say you are uncertain and ask for clarification. "
     "Follow the output format instructions exactly."),
    ("human",
     "Question:\n{question}\n\nContext:\n{context}\n\nOutput format:\n{format_instructions}")
])

def graph_qa_chain(question: str) -> Dict[str, Any]:
    """
    Retrieve top-k chunks from Neo4j vector index, ask the LLM for a concise answer,
    and return strict JSON with keys:
      - text: the concise answer
      - refrence: list of 'Chapter N Verse M' derived from retrieved docs (no hallucinations)

    We keep 'refrence' spelling as requested.
    """
    docs: List[Document] = retriever.get_relevant_documents(question)
    context = format_context(docs)
    refs = extract_refs(docs)  # authoritative list of references from metadata/content

    # Ask the LLM to produce only {"text": "..."} JSON
    result: Dict[str, Any] = (prompt | llm | parser).invoke({
        "question": question,
        "context": context,
        "format_instructions": format_instructions
    })

    # Inject the clean references; override anything the model might have added (if any)
    result["refrence"] = refs
    return result

# Example:
# out = graph_qa_chain("Why did Arjuna ask Krishna to place the chariot between the armies?")
# out -> {"text": "...", "refrence": ["Chapter 1 Verse 21", "Chapter 1 Verse 22"]}

In [22]:
response = graph_qa_chain("Why did Arjuna ask Krishna to place the chariot between the armies?")
print(response)

{'text': "Arjuna requested Krishna to position the chariot between the armies so he could clearly identify his opponents before the battle commenced. He wanted to see who he would be facing and understand the full scope of the conflict. Arjuna's request stemmed from a desire to know his enemies and prepare himself mentally for the impending battle.", 'refrence': ['Chapter 1 Verse 19', 'Chapter 1 Verse 20', 'Chapter 1 Verse 21', 'Chapter 1 Verse 22', 'Chapter 1 Verse 23', 'Chapter 1 Verse 24', 'Chapter 1 Verse 25', 'Chapter 1 Verse 26', 'Chapter 1 Verse 27', 'Chapter 1 Verse 28', 'Chapter 1 Verse 29', 'Chapter 1 Verse 30', 'Chapter 1 Verse 43', 'Chapter 1 Verse 44', 'Chapter 1 Verse 45', 'Chapter 1 Verse 46', 'Chapter 1 Verse 47', 'Chapter 2 Verse 1', 'Chapter 11 Verse 31', 'Chapter 11 Verse 32', 'Chapter 11 Verse 33', 'Chapter 11 Verse 34', 'Chapter 11 Verse 35', 'Chapter 11 Verse 36', 'Chapter 18 Verse 69', 'Chapter 18 Verse 70', 'Chapter 18 Verse 71', 'Chapter 18 Verse 72', 'Chapter 